In [2]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


#DATABASE CONNECTION

In [3]:
conn = sqlite3.connect("expense_tracker.db")
cursor = conn.cursor()

print('Database Created Successfully')

Database Created Successfully


In [4]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS expenses (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    date TEXT,
    category TEXT,
    amount REAL,
    expense_type TEXT,
    forgiven INTEGER
)
""")

conn.commit()
print("Table created successfully")


Table created successfully


In [5]:
cursor.execute(" select name from sqlite_master where type = 'table' ")
print(cursor.fetchall())

[('expenses',), ('sqlite_sequence',)]


In [6]:
%%writefile requirements.txt
pandas
numpy
matplotlib


Overwriting requirements.txt
